# Transformation and Load (With Visualization)

In [1]:
# Installing Libraries
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import numpy as np

### 1. Read Data

##### 1.1 Read Temperature Data

In [2]:
temp_data=pd.read_excel("variables.xlsx",sheet_name='Temperatura')
temp_data.head(5)


,Time,Temp
0,2025-02-06 00:00:00,14.04
1,2025-02-06 00:05:00,14.12
2,2025-02-06 00:10:00,14.12
3,2025-02-06 00:15:00,14.20
4,2025-02-06 00:20:00,14.32


##### 1.2 Read Humidity Data

In [3]:
hum_data=pd.read_excel("variables.xlsx",sheet_name='Humedad')
hum_data.head(5)

,Time,Hum
0,2025-02-06 00:00:00,86.044098
1,2025-02-06 00:05:00,86.344389
2,2025-02-06 00:10:00,86.468626
3,2025-02-06 00:15:00,86.701349
4,2025-02-06 00:20:00,87.072792


##### 1.3 Read Radiation Data

In [4]:
rad_data=pd.read_excel("variables.xlsx",sheet_name='Radiacion')
rad_data.head(5)

,Time,PPFD
0,2025-02-06 00:00:00,0.0
1,2025-02-06 00:05:00,0.0
2,2025-02-06 00:10:00,0.0
3,2025-02-06 00:15:00,0.0
4,2025-02-06 00:20:00,0.0


##### 1.3 Read Wind Data

In [5]:
wind_data=pd.read_excel("variables.xlsx",sheet_name='Viento')
wind_data.head(5)

,Time,Direccion,Velocidad
0,2025-02-06 00:00:00,177,0.07
1,2025-02-06 00:05:00,187,0.06
2,2025-02-06 00:10:00,210,0.08
3,2025-02-06 00:15:00,225,0.06
4,2025-02-06 00:20:00,352,0.06


### 2. Transformation

##### 2.1 Temperature Transformation

2.1.1 Mean, Minimum and Maximum temperature

In [6]:
# Set the time variable
temp_data['Time']=pd.to_datetime(temp_data['Time'])

# To get average, Minimum and Maximum
temp_data.set_index('Time',inplace=True)

#Resample by day calculating  min, max and average
temp_data_day=temp_data.resample('D').agg({
    'Temp':['mean','min','max']
})
temp_data_day.head(5)

Temp              
                 mean    min    max
Time                               
2025-02-06  20.008194  11.70  35.54
2025-02-07  20.867222  12.60  38.58
2025-02-08  18.227778  12.38  37.06
2025-02-09  15.745417  12.58  20.68
2025-02-10  15.756944  11.98  24.72

2.1.2 GDD

In [7]:
tbase=10
temp_data_day[('Temp', 'GDD')]=((temp_data_day[('Temp', 'min')]+temp_data_day[('Temp', 'max')])/2)-tbase
temp_data_day[('Temp', 'GDD_cum')]=temp_data_day[('Temp','GDD')].cumsum()
temp_data_day

Temp                              
                 mean    min    max    GDD  GDD_cum
Time                                               
2025-02-06  20.008194  11.70  35.54  13.62    13.62
2025-02-07  20.867222  12.60  38.58  15.59    29.21
2025-02-08  18.227778  12.38  37.06  14.72    43.93
2025-02-09  15.745417  12.58  20.68   6.63    50.56
2025-02-10  15.756944  11.98  24.72   8.35    58.91
...               ...    ...    ...    ...      ...
2025-05-31  17.116319  10.90  29.60  10.25  1153.20
2025-06-01  17.839583  12.80  30.10  11.45  1164.65
2025-06-02  17.248958  11.50  30.50  11.00  1175.65
2025-06-03  16.491319  10.20  25.70   7.95  1183.60
2025-06-04  14.386290  11.50  21.40   6.45  1190.05

[119 rows x 5 columns]

##### 2.2 Humidity Transformation 

2.2.1 Mean relative humidity

In [8]:
# Set the time variable
hum_data['Time']=pd.to_datetime(hum_data['Time'])

# To get average
hum_data.set_index('Time',inplace=True)

#Resample by day calculating mean
hum_data_day=hum_data.resample('D').agg({
    'Hum':['mean']
})
hum_data_day.head(5)

,Hum
,mean
Time,
2025-02-06,64.348827
2025-02-07,64.066796
2025-02-08,69.215451
2025-02-09,82.138615
2025-02-10,82.659014


2.1.1 VPD --> Vapor Pressure Deficit

In [9]:
# Vapor de saturación (es)
t=temp_data_day[('Temp',    'mean')]
es=0.6108*np.exp((17.27*t)/(t+237.3))
# Presion de vapor real
ea=es*hum_data_day[('Hum',    'mean')]/100

vpd=pd.DataFrame({'vpd':es-ea})
vpd.head(5)


,vpd
Time,
2025-02-06,0.834048
2025-02-07,0.886398
2025-02-08,0.644547
2025-02-09,0.319528
2025-02-10,0.310447


##### 2.3 Radiation Transformation

2.3.1 DLI

In [10]:
# Set the time variable
rad_data['Time']=pd.to_datetime(rad_data['Time'])
# Extract the day of the date 
rad_data['Date'] = rad_data['Time'].dt.date

# Interval --> 5 minutes = 300 seconds
interval=300
# Grouping by day and calculate DLI
dli_diario = rad_data.groupby('Date')['PPFD'].sum() * interval / 1_000_000
dli_diario = dli_diario.reset_index(name='DLI')
dli_diario

,Date,DLI
0,2025-02-06,20.441986
1,2025-02-07,25.024493
2,2025-02-08,14.404449
3,2025-02-09,12.197076
4,2025-02-10,10.022443
...,...,...
114,2025-05-31,6.803610
115,2025-06-01,7.974330
116,2025-06-02,7.922160
117,2025-06-03,5.863560


2.3.2 Cumulative DLI

In [11]:
dli_diario['cum_DLI']=dli_diario['DLI'].cumsum()

##### 2.4 Wind Transformation 

2.4.1 Velocity

In [12]:
# Set the time variable
wind_data['Time']=pd.to_datetime(wind_data['Time'])

# To get average
wind_data.set_index('Time',inplace=True)

#Resample by day calculating mean
wind_data_day=wind_data.resample('D').agg({
    'Velocidad':['mean']
})
wind_data_day.head(5)

,Velocidad
,mean
Time,
2025-02-06,0.191493
2025-02-07,0.218750
2025-02-08,0.405868
2025-02-09,0.486354
2025-02-10,0.426076


### 3. Load and Visualization

##### Temperature Visualizations

In [13]:
fig_temp = go.Figure()

# Now x = df.index
fig_temp.add_trace(go.Scatter(x=temp_data_day.index, y=temp_data_day[('Temp', 'mean')], mode='lines+markers', name='Mean Temp'))
fig_temp.add_trace(go.Scatter(x=temp_data_day.index, y=temp_data_day[('Temp', 'min')], mode='lines+markers', name='Min Temp'))
fig_temp.add_trace(go.Scatter(x=temp_data_day.index, y=temp_data_day[('Temp', 'max')], mode='lines+markers', name='Max Temp'))

# Add titles and labels
fig_temp.update_layout(
    title='Daily Temperature Summary',
    xaxis_title='Time',
    yaxis_title='Temperature (°C)',
    template='plotly_dark'
)

# Show the plot
fig_temp.show()

In [14]:
fig_gdd = go.Figure()

# Now x = df.index
fig_gdd.add_trace(go.Scatter(x=temp_data_day.index, y=temp_data_day[('Temp', 'GDD_cum')], 
                         mode='lines+markers', 
                         name='Mean GDD',
                         line=dict(color='#FFA500'),
                         marker=dict(color='#FFA500')))

# Add titles and labels
fig_gdd.update_layout(
    title='Cumulative GDD Summary per day',
    xaxis_title='Time',
    yaxis_title='GDD (C°)',
    template='plotly_dark'
)

# Show the plot
fig_gdd.show()

In [15]:
# Export temperature plots
## Minimum, Maximum and Mean temperature
fig_temp.write_html("/Users/jorgeandresjolahernandez/Desktop/Produccion_Hortalizas/tomato_climate_project/static/visualizations/temp_day.html",full_html=False,include_plotlyjs='cdn')
fig_gdd.write_html("/Users/jorgeandresjolahernandez/Desktop/Produccion_Hortalizas/tomato_climate_project/static/visualizations/gdd_cum.html")


##### Humidity Visualizations

In [16]:
fig_hum = go.Figure()

# Now x = df.index
fig_hum.add_trace(go.Scatter(x=hum_data_day.index, y=hum_data_day[('Hum', 'mean')], 
                         mode='lines+markers', 
                         name='Mean Hum',
                         line=dict(color='#CCCC00'),
                         marker=dict(color='#CCCC00')))

# Add titles and labels
fig_hum.update_layout(
    title='Daily Humidity Summary',
    xaxis_title='Time',
    yaxis_title='Humidity (%rh)',
    template='plotly_dark'
)

# Show the plot
fig_hum.show()

In [17]:
fig_vpd = go.Figure()

# Now x = df.index
fig_vpd.add_trace(go.Scatter(x=vpd.index, y=vpd['vpd'], 
                         mode='lines+markers', 
                         name='Mean Hum',
                         line=dict(color='#CCCC00'),
                         marker=dict(color='#CCCC00')))

# Add titles and labels
fig_vpd.update_layout(
    title='Vapor Pressure Deficit (VPD)',
    xaxis_title='Time',
    yaxis_title='VPD (kPa)',
    template='plotly_dark'
)

# Show the plot
fig_vpd.show()

In [18]:
# Export humidity plots
fig_hum.write_html("/Users/jorgeandresjolahernandez/Desktop/Produccion_Hortalizas/tomato_climate_project/static/visualizations/hum_day.html")
fig_vpd.write_html("/Users/jorgeandresjolahernandez/Desktop/Produccion_Hortalizas/tomato_climate_project/static/visualizations/vpd_day.html")

##### Radation Visualizations

In [19]:
fig_dli = go.Figure()

# Now x = df.index
fig_dli.add_trace(go.Scatter(x=dli_diario['Date'], y=dli_diario['DLI'], 
                         mode='lines+markers', 
                         name='DLI',
                         line=dict(color='#8B4513'),
                         marker=dict(color='#8B4513')))

# Add titles and labels
fig_dli.update_layout(
    title='Daily DLI Summary',
    xaxis_title='Time',
    yaxis_title='DLI (mol/m²/día)',
    template='plotly_dark'
)

# Show the plot
fig_dli.show()

In [20]:
fig_cumDLI = go.Figure()

# Now x = df.index
fig_cumDLI.add_trace(go.Scatter(x=dli_diario['Date'], y=dli_diario['cum_DLI'], 
                         mode='lines+markers', 
                         name='Mean GDD',
                         line=dict(color='#8B4513'),
                         marker=dict(color='#8B4513')))

# Add titles and labels
fig_cumDLI.update_layout(
    title='Cumulative DLI Summary per day',
    xaxis_title='Time',
    yaxis_title='DLI (mol/m²/día)',
    template='plotly_dark'
)

# Show the plot
fig_cumDLI.show()

In [21]:
# Export Radiation plots
fig_dli.write_html("/Users/jorgeandresjolahernandez/Desktop/Produccion_Hortalizas/tomato_climate_project/static/visualizations/dli_day.html")
fig_cumDLI.write_html("/Users/jorgeandresjolahernandez/Desktop/Produccion_Hortalizas/tomato_climate_project/static/visualizations/fig_cumDLI.html")

##### Wind Visualizations

In [22]:
fig_wind = go.Figure()

# Now x = df.index
fig_wind.add_trace(go.Scatter(x=wind_data_day.index, y=wind_data_day[('Velocidad', 'mean')], 
                         mode='lines+markers', 
                         name='Mean Velocidad',
                         line=dict(color='white'),
                         marker=dict(color='white')))

# Add titles and labels
fig_wind.update_layout(
    title='Daily Wind Speed Summary',
    xaxis_title='Time',
    yaxis_title='Speed (m/s)',
    template='plotly_dark'
)

# Show the plot
fig_wind.show()

In [23]:
fig_wind.write_html("/Users/jorgeandresjolahernandez/Desktop/Produccion_Hortalizas/tomato_climate_project/static/visualizations/fig_wind.html")